# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise

In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [16]:
data = pd.read_csv("assignment3_creditcard.csv")

In [17]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28678 entries, 0 to 28677
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      28678 non-null  float64
 1   V2      28678 non-null  float64
 2   V3      28678 non-null  float64
 3   V4      28678 non-null  float64
 4   V5      28678 non-null  float64
 5   V6      28678 non-null  float64
 6   V7      28678 non-null  float64
 7   V8      28678 non-null  float64
 8   V9      28678 non-null  float64
 9   V10     28678 non-null  float64
 10  V11     28678 non-null  float64
 11  V12     28678 non-null  float64
 12  V13     28678 non-null  float64
 13  V14     28678 non-null  float64
 14  V15     28678 non-null  float64
 15  V16     28678 non-null  float64
 16  V17     28678 non-null  float64
 17  V18     28678 non-null  float64
 18  V19     28678 non-null  float64
 19  V20     28678 non-null  float64
 20  V21     28678 non-null  float64
 21  V22     28678 non-null  float64
 22

In [19]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
V1,28678.0,-0.015438,2.031529,-30.552380,-0.916927,0.020050,1.317461,2.399484
V2,28678.0,0.053653,1.616186,-42.172688,-0.575381,0.075358,0.806957,21.467203
V3,28678.0,-0.046031,1.758169,-31.103685,-0.899872,0.180610,1.029928,4.069865
V4,28678.0,0.037348,1.482109,-5.560118,-0.843321,-0.008844,0.771958,11.927512
V5,28678.0,-0.033724,1.486998,-42.147898,-0.714901,-0.060040,0.613328,32.911462
V6,28678.0,-0.003299,1.339259,-21.929312,-0.763757,-0.271363,0.397269,22.529298
V7,28678.0,-0.051054,1.454827,-41.506796,-0.568146,0.036107,0.559409,36.677268
V8,28678.0,0.006064,1.364342,-39.267378,-0.206103,0.022463,0.329606,20.007208
V9,28678.0,-0.018530,1.134065,-13.434066,-0.661909,-0.055095,0.605704,8.113152
V10,28678.0,-0.041149,1.252593,-24.403185,-0.543450,-0.097390,0.460681,15.236028


In [20]:
data = data.dropna(axis=0)

In [21]:
data.isnull().sum()

V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
V11      0
V12      0
V13      0
V14      0
V15      0
V16      0
V17      0
V18      0
V19      0
V20      0
V21      0
V22      0
V23      0
V24      0
V25      0
V26      0
V27      0
V28      0
Class    0
dtype: int64

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [23]:
# 데이터를 특성(X)과 타겟(y)으로 분리합니다
X = data.drop('Class', axis=1)
y = data['Class']

In [24]:
# 데이터를 학습 데이터와 테스트 데이터로 분리합니다
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델을 학습시킵니다
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

LogisticRegression()

In [25]:
# 모델의 성능을 평가합니다
y_pred = log_reg.predict(X_test)
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
print('Precision:', metrics.precision_score(y_test, y_pred))
print('Recall:', metrics.recall_score(y_test, y_pred))
print('F1 Score:', metrics.f1_score(y_test, y_pred))


Accuracy: 0.9987796373779637
Precision: 1.0
Recall: 0.8541666666666666
F1 Score: 0.9213483146067416


In [26]:
# 모델의 성능을 개선하기 위해 규제 강도를 조정해봅니다
log_reg2 = LogisticRegression(C=0.5)
log_reg2.fit(X_train, y_train)


LogisticRegression(C=0.5)

In [27]:
# 개선된 모델의 성능을 평가합니다
y_pred2 = log_reg2.predict(X_test)
print('Improved Accuracy:', metrics.accuracy_score(y_test, y_pred2))
print('Improved Precision:', metrics.precision_score(y_test, y_pred2))
print('Improved Recall:', metrics.recall_score(y_test, y_pred2))
print('Improved F1 Score:', metrics.f1_score(y_test, y_pred2))

Improved Accuracy: 0.9987796373779637
Improved Precision: 1.0
Improved Recall: 0.8541666666666666
Improved F1 Score: 0.9213483146067416
